In [1]:
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler
import numpy as np
import pandas as pd
import glob
import cv2
import os

In [2]:
def loadHouseAttributes(inputPath):
    cols = ["bedrooms", "bathrooms", "area", "zipcode", "price"]
    df = pd.read_csv(inputPath, sep=" ", header=None, names=cols)
    
    zipcodes = df['zipcode'].value_counts().keys().tolist()
    counts = df['zipcode'].value_counts().tolist()
    
    for (zipcode, count) in zip(zipcodes, counts):
        if count < 25:
            idxs = df[df['zipcode'] == zipcode].index
            df.drop(idxs, inplace=True)
            
    return df

In [3]:
def processHouseAttributes(df, train, test):
    
    continuous = ['bedrooms', 'bathrooms', 'area']
    
    cs = MinMaxScaler()
    trainContinuous = cs.fit_transform(train[continuous])
    testContinuous = cs.transform(test[continuous])
    
    zipBinarizer = LabelBinarizer().fit(df['zipcode'])
    trainCategorical = zipBinarizer.transform(train['zipcode'])
    testCategorical = zipBinarizer.transform(test['zipcode'])
    
    trainX = np.hstack([trainCategorical, trainContinuous])
    testX = np.hstack([testCategorical, testContinuous])
    
    return (trainX, testX)

In [4]:
def loadHouseImages(df, inputPath):
    
    images = []
    
    for i in df.index.values:
        
        basePath = os.path.sep.join([inputPath, "{}_*".format(i+1)])
        housePaths = sorted(list(glob.glob(basePath)))
        
        inputImages = []
        outputImage = np.zeros((64, 64, 3), dtype='uint8')
        
        for housePath in housePaths:
            
            image = cv2.imread(housePath)
            image = cv2.resize(image, (32, 32))
            inputImages.append(image)
            
        outputImage[0:32, 0:32] = inputImages[0]
        outputImage[0:32, 32:64] = inputImages[1]
        outputImage[32:64, 32:64] = inputImages[2]
        outputImage[32:64, 0:32] = inputImages[3]
        
        images.append(outputImage)
        
    return np.array(images)

In [5]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model

/home/praneet/envs/deeplearning/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
def create_mlp(dim, regress=False):
    
    model = Sequential()
    model.add(Dense(8, input_dim = dim, activation='relu'))
    model.add(Dense(4, activation='relu'))
    
    if regress:
        model.add(Dense(1, activation='linear'))
        
    return model

In [7]:
def create_cnn(width, height, depth, filters=(16, 32, 64), regress=False):
    
    inputShape = (height, width, depth)
    chanDim = -1
    
    inputs = Input(shape=inputShape)
    
    for (i, f) in enumerate(filters):
        
        if i == 0:
            x = inputs
            
        x = Conv2D(f, (3, 3), padding='same')(x)
        x = Activation('relu')(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        
        x = Flatten()(x)
        x = Dense(16)(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Dropout(0.5)(x)
        
        x = Dense(4)(x)
        x = Activation('relu')(x)
        
        if regress:
            x = Dense(1, activation='linear')(x)
            
        model = Model(inputs, x)
        
        return model

In [8]:
from sklearn.model_selection import train_test_split
from keras.layers.core import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import concatenate
import numpy as np
import argparse
import locale
import os

In [9]:
inputPath = os.path.sep.join(['dataset', "HousesInfo.txt"])
df = loadHouseAttributes(inputPath)

images = loadHouseImages(df, 'dataset/Houses Dataset/')
images = images / 255.0

In [10]:
split = train_test_split(df, images, test_size=0.25, random_state=42)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

In [11]:
maxPrice = trainAttrX["price"].max()
trainY = trainAttrX["price"] / maxPrice
testY = testAttrX["price"] / maxPrice

In [12]:
(trainAttrX, testAttrX) = processHouseAttributes(df, trainAttrX, testAttrX)

In [13]:
mlp = create_mlp(trainAttrX.shape[1], regress=False)
cnn = create_cnn(64, 64, 3, regress=False)

In [14]:
combinedInput = concatenate([mlp.output, cnn.output])

In [15]:
x = Dense(4, activation="relu")(combinedInput)
x = Dense(1, activation="linear")(x)

In [16]:
model = Model(inputs=[mlp.input, cnn.input], outputs=x)

In [17]:
opt = Adam(lr=1e-3, decay=1e-3 / 200)

In [18]:
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

In [19]:
model.fit([trainAttrX, trainImagesX], trainY,
          validation_data=([testAttrX, testImagesX], testY),
          epochs=200, batch_size=8)

Train on 271 samples, validate on 91 samples
Epoch 1/200
271/271 [==============================] - 3s 10ms/step - loss: 387.2413 - val_loss: 403.6908
Epoch 2/200
271/271 [==============================] - 2s 7ms/step - loss: 152.7092 - val_loss: 390.3597
Epoch 3/200
271/271 [==============================] - 2s 7ms/step - loss: 95.9997 - val_loss: 93.6528
Epoch 4/200
271/271 [==============================] - 2s 7ms/step - loss: 89.0639 - val_loss: 78.8276
Epoch 5/200
271/271 [==============================] - 2s 7ms/step - loss: 79.1172 - val_loss: 76.8382
Epoch 6/200
271/271 [==============================] - 2s 7ms/step - loss: 77.2598 - val_loss: 96.7996
Epoch 7/200
271/271 [==============================] - 2s 7ms/step - loss: 68.1413 - val_loss: 262.6899
Epoch 8/200
271/271 [==============================] - 2s 7ms/step - loss: 69.7665 - val_loss: 974.3186
Epoch 9/200
271/271 [==============================] - 2s 7ms/step - loss: 68.7231 - val_loss: 783.3896
Epoch 10/200
271/271

271/271 [==============================] - 2s 7ms/step - loss: 26.5063 - val_loss: 29.0102
Epoch 80/200
271/271 [==============================] - 2s 7ms/step - loss: 27.2416 - val_loss: 27.9770
Epoch 81/200
271/271 [==============================] - 2s 7ms/step - loss: 24.9409 - val_loss: 29.9294
Epoch 82/200
271/271 [==============================] - 2s 7ms/step - loss: 26.5752 - val_loss: 27.8239
Epoch 83/200
271/271 [==============================] - 2s 7ms/step - loss: 25.6054 - val_loss: 27.7396
Epoch 84/200
271/271 [==============================] - 2s 7ms/step - loss: 24.4691 - val_loss: 27.5873
Epoch 85/200
271/271 [==============================] - 2s 7ms/step - loss: 25.3166 - val_loss: 31.0219
Epoch 86/200
271/271 [==============================] - 2s 7ms/step - loss: 26.0402 - val_loss: 28.3973
Epoch 87/200
271/271 [==============================] - 2s 8ms/step - loss: 25.6433 - val_loss: 27.4929
Epoch 88/200
271/271 [==============================] - 2s 8ms/step - loss: 2

271/271 [==============================] - 2s 7ms/step - loss: 21.7530 - val_loss: 28.1577
Epoch 158/200
271/271 [==============================] - 2s 7ms/step - loss: 21.8505 - val_loss: 27.1567
Epoch 159/200
271/271 [==============================] - 2s 7ms/step - loss: 21.7350 - val_loss: 27.0707
Epoch 160/200
271/271 [==============================] - 2s 7ms/step - loss: 23.1395 - val_loss: 27.1502
Epoch 161/200
271/271 [==============================] - 2s 7ms/step - loss: 21.5551 - val_loss: 28.9963
Epoch 162/200
271/271 [==============================] - 2s 8ms/step - loss: 22.7337 - val_loss: 28.8399
Epoch 163/200
271/271 [==============================] - 2s 7ms/step - loss: 22.1793 - val_loss: 29.7456
Epoch 164/200
271/271 [==============================] - 2s 7ms/step - loss: 23.2807 - val_loss: 27.1080
Epoch 165/200
271/271 [==============================] - 2s 7ms/step - loss: 22.2932 - val_loss: 27.0398
Epoch 166/200
271/271 [==============================] - 2s 8ms/step 

In [20]:
preds = model.predict([testAttrX, testImagesX])

In [22]:
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)

# compute the mean and standard deviation of the absolute percentage
# difference
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)
 
# finally, show some statistics on our model
locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
print("[INFO] avg. house price: {}, std house price: {}".format(
    locale.currency(df["price"].mean(), grouping=True),
    locale.currency(df["price"].std(), grouping=True)))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

[INFO] avg. house price: $533,388.27, std house price: $493,403.08
[INFO] mean: 28.24%, std: 21.57%
